In [1]:
import pandas as pd
import os
os.chdir("/group-volume/binfeng/wsdm/stage_distill")
from utils import *

import numpy as np

def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits))  # Subtract max for numerical stability
    return exp_logits / np.sum(exp_logits)

def pred(x):
    logits = np.array([x.logits_model_a, x.logits_model_b])
    probs = softmax(logits)
    
    prob_model_a = probs[0]
    prob_model_b = probs[1]
    
    if prob_model_a > 0.5:
        return "model_a"
    else:
        return "model_b"
def merge_logits(x):
    return [x.logits_model_a, x.logits_model_b]

## HFOPEN and LMSYS

In [2]:
hfopen1 = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwen32b_plabel_hfopen.csv")
hfopen2 = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwencd32b_plabel_hfopen.csv")
lmsys1 = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwen32b_plabel_lmsys.csv")
lmsys2 = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwencd32b_plabel_lmsys.csv")


In [3]:
hfopen_m = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwen32b_plabel_hfopen.csv")
lmsys_m = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwen32b_plabel_lmsys.csv")
hfopen_m["logits_model_a"] = 0.4 * hfopen1["logits_model_a"] + 0.6 * hfopen2["logits_model_a"]
lmsys_m["logits_model_a"] = 0.4 * lmsys1["logits_model_a"] + 0.6 * lmsys2["logits_model_a"]
hfopen_m["logits_model_b"] = 0.4 * hfopen1["logits_model_b"] + 0.6 * hfopen2["logits_model_b"]
lmsys_m["logits_model_b"] = 0.4 * lmsys1["logits_model_b"] + 0.6 * lmsys2["logits_model_b"]

In [4]:
hfopen_m["winner"] = hfopen1.apply(pred, axis=1)
lmsys_m["winner"] = lmsys1.apply(pred, axis=1)

In [7]:
# hfopen = hfopen1[hfopen_m["winner"] != "tie"]
# lmsys = lmsys1[lmsys1["winner"] != "tie"]
# print(hfopen.shape, lmsys.shape)

## PPT full

In [8]:
# ppt
hfopen = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwen32b_plabel_hfopen.csv")
lmsys = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwen32b_plabel_lmsys.csv")
hfopen["winner"] = hfopen_m["winner"]
lmsys["winner"] = lmsys_m["winner"]
ppt = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwen32b_plabel_ppt.csv")
ppt_full = pd.concat([hfopen, lmsys, ppt], axis=0)

In [9]:
ppt_full["logits_qwen32"] = ppt_full.apply(merge_logits, axis=1)
ppt_full = ppt_full[['prompt', 'response_a', 'response_b', 'winner', 'logits_qwen32']]

In [10]:
ppt_full.dropna(inplace=True)
ppt_full.reset_index(drop=True, inplace=True)
ppt_full.shape

(196794, 5)

In [13]:
ppt_full.to_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ppt135k.parquet", index=False)

## FT

In [11]:
# ppt
ft = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwen32b_plabel_kaggle.csv")

In [12]:
ft["logits_qwen32"] = ft.apply(merge_logits, axis=1)
ft = ft[['prompt', 'response_a', 'response_b', 'winner', 'language', 'logits_qwen32']]
ft.dropna(inplace=True)
ft.reset_index(drop=True)
ft.shape

(48437, 6)

In [14]:
ft.to_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ft48k.parquet", index=False)

## FT (2)

In [15]:
ft = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwencd32b_plabel_kaggle.csv")
ft["logits_qwencd"] = ft.apply(merge_logits, axis=1)
ft = ft[['prompt', 'response_a', 'response_b', 'winner', 'language', 'logits_qwencd']]
ft.dropna(inplace=True)
ft.reset_index(drop=True)
ft.shape

(48437, 6)

In [16]:
old_ft = pd.read_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ft48k.parquet")

In [17]:
old_ft["logits_qwencd"] = ft["logits_qwencd"]
old_ft.to_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ft48k.parquet", index=False)

## PPT2

In [19]:
# ppt
hfopen = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwencd32b_plabel_hfopen.csv")
lmsys = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwencd32b_plabel_lmsys.csv")
hfopen["winner"] = hfopen_m["winner"]
lmsys["winner"] = lmsys_m["winner"]
ppt = pd.read_csv("/group-volume/binfeng/wsdm/stage_distill/plabels/qwencd32b_plabel_ppt.csv")
ppt_full = pd.concat([hfopen, lmsys, ppt], axis=0)

In [20]:
ppt_full["logits_qwencd"] = ppt_full.apply(merge_logits, axis=1)
ppt_full = ppt_full[['prompt', 'response_a', 'response_b', 'winner', 'logits_qwencd']]

In [21]:
ppt_full.dropna(inplace=True)
ppt_full = ppt_full.reset_index(drop=True)
ppt_full.shape

(196794, 5)

In [22]:
ppt_old = pd.read_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ppt135k.parquet")

In [24]:
ppt_full["logits_qwen32"]  = ppt_old["logits_qwen32"]

In [25]:
ppt_full.to_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ppt135k.parquet", index=False)

## Clean outliers

In [26]:
ppt = pd.read_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ppt135k.parquet")
ft = pd.read_parquet("/group-volume/binfeng/wsdm/stage_distill/datap/ft48k.parquet")

In [40]:
def prob(x):
    logits = np.array(np.mean([x["logits_qwencd"], x["logits_qwen32"]], axis=0))
    probs = softmax(logits)
    prob_model_a = probs[0]
    # prob_model_b = probs[1]
    return prob_model_a

ppt["a_win"] = ppt.apply(prob, axis=1)
ft["a_win"] = ft.apply(prob, axis=1)

In [72]:
def isoutlier(x):
    if x["winner"]=="model_a" and x["a_win"] < 0.3:
        return True
    if x["winner"]=="model_b" and x["a_win"] > 0.7:
        return True
    return False

ppt["outlier"] = ppt.apply(isoutlier, axis=1)
ft["outlier"] = ft.apply(isoutlier, axis=1)

In [78]:
def calibrate(outlier, logits):
    l = logits.copy()
    if outlier:
        l[0], l[1] = 0.5, 0.5
        return l
    else:
        return l
    
ppt["logits_qwencd_cali"] = ppt.apply(lambda x: calibrate(x["outlier"], x["logits_qwencd"]), axis=1)
ppt["logits_qwen32_cali"] = ppt.apply(lambda x: calibrate(x["outlier"], x["logits_qwen32"]), axis=1)
ft["logits_qwencd_cali"] = ft.apply(lambda x: calibrate(x["outlier"], x["logits_qwencd"]), axis=1)
ft["logits_qwen32_cali"] = ft.apply(lambda x: calibrate(x["outlier"], x["logits_qwen32"]), axis=1)

In [83]:
ft.drop(['a_win', 'outlier'], axis=1, inplace=True)
ppt.drop(['a_win', 'outlier'], axis=1, inplace=True)

In [84]:
ft.shape

(48437, 10)

In [85]:
ppt.shape

(196794, 8)

In [87]:
ft.to_parquet("/group-volume/binfeng/wsdm/stage_final/data/ft48k_calibrated.parquet")
ppt.to_parquet("/group-volume/binfeng/wsdm/stage_final/data/ppt196k_calibrated.parquet")

In [88]:
ft

,prompt,response_a,response_b,winner,language,logits_qwen32,logits_qwencd,len,logits_qwencd_cali,logits_qwen32_cali
0,draw a unicorn using asciii,Sure! Here's a simple ASCII art representation...,Sure! Here’s a simple ASCII art representation...,model_a,English,"[1.9086914, -1.5205078]","[1.9257812, -1.4804688]",-101658,"[1.9257812, -1.4804688]","[1.9086914, -1.5205078]"
1,"Сделать декомпозицию процесса ""выполнить ремон...","## Декомпозиция процесса ""Выполнить ремонт"" (D...","## Декомпозиция процесса ""Выполнить ремонт"" (д...",model_b,Russian,"[-1.7275391, 2.0708008]","[-1.5664062, 1.953125]",22762,"[-1.5664062, 1.953125]","[-1.7275391, 2.0708008]"
2,NOMBRE DE LA ACTIVIDAD: Análisis descriptivo t...,"En el proceso de liofilización de alimentos, p...",## Análisis termodinámico de la liofilización:...,model_a,Spanish,"[1.8520508, -1.4663086]","[1.7304688, -1.3203125]",-52604,"[1.7304688, -1.3203125]","[1.8520508, -1.4663086]"
3,find the logic error in this code and explain ...,The logic error in the provided code is relate...,The code is well-structured and clean. However...,model_a,English,"[0.321167, 0.15768433]","[0.36328125, 0.032226562]",8424,"[0.36328125, 0.032226562]","[0.321167, 0.15768433]"
4,"\nДобрый день, коллега. От Вас требуется завер...",```python\n \n \n \n \n \n \...,"В вашем коде есть несколько импортов, которые ...",model_b,Russian,"[-0.9765625, 1.4189453]","[-0.7675781, 1.1757812]",13697,"[-0.7675781, 1.1757812]","[-0.9765625, 1.4189453]"
...,...,...,...,...,...,...,...,...,...,...
48432,摩挲着手指甲，左右手交替进行。\n\n强行保持镇定，素世微笑试图挣扎道：\n\n“清，那个是...,她的手指不由自主地在指甲上来回摩挲着，双手交替进行，仿佛这样能缓解心底涌起的紧张。\n\n素...,素世轻轻摩挲着修长的手指，左手与右手交替，指尖在微光中闪烁着微弱的光芒。她强作镇定，嘴角勉强...,model_b,Chinese,"[0.45690918, 0.085754395]","[0.256073, 0.17407227]",-49,"[0.256073, 0.17407227]","[0.45690918, 0.085754395]"
48433,一名银行软件开发中心技术平台部门的高级经理参加党的二十大三中全会研讨会，针对主题“深入学习贯...,尊敬的各位领导、同事：\n\n大家好！今天，我很荣幸能够参加党的二十大三中全会研讨会，并就“...,尊敬的各位领导、各位同事：\n\n大家好！\n\n今天，我们怀着无比激动的心情，在这里召开党...,model_b,Chinese,"[-0.30456543, 0.8005371]","[-0.6660156, 1.0820312]",-566,"[-0.6660156, 1.0820312]","[-0.30456543, 0.8005371]"
48434,葡萄糖是从葡萄中提取的吗,不是的。葡萄糖的名字可能会让人误以为它是从葡萄中提取的，但实际上，葡萄糖是一种单糖，也被称为...,葡萄糖并不是专门从葡萄中提取的，尽管它的名字听起来像是与葡萄有直接的联系。实际上，葡萄糖是一...,model_b,Chinese,"[-0.2420044, 0.7753906]","[-0.140625, 0.5571289]",-36,"[-0.140625, 0.5571289]","[-0.2420044, 0.7753906]"
48435,关于道家化童子法事的利益点写一段文案,道家化童子法事，源远流长，承载着深厚的文化与精神性价值。这一神秘而古老的仪式，不仅是身心净化...,道家化童子法事，旨在化解童子命带来的诸多不利影响，助您开启顺遂人生。童子命之人常遇坎坷，诸事...,model_b,Chinese,"[-0.7722168, 1.234375]","[-0.45214844, 0.8769531]",94,"[-0.45214844, 0.8769531]","[-0.7722168, 1.234375]"
